# Sudoku Optimization Model 

## Model

### Decision Variables
Let $x_{ijk} \in \{0,1\}$ where:
- $i, j \in \{1, 2, ..., 9\}$ represent row and column indices
- $k \in \{1, 2, ..., 9\}$ represents the digit value
- $x_{ijk} = 1$ if cell $(i, j)$ contains digit $k$, otherwise $x_{ijk} = 0$

### Objective Function
**No objective function** - This is a **feasibility problem**
- Goal: Find any solution that satisfies all constraints
- No need to maximize or minimize anything

### Constraints

#### 1. Each cell contains exactly one digit
$$\sum_{k=1}^{9} x_{ijk} = 1 \quad \forall i, j \in \{1, ..., 9\}$$

#### 2. Each digit appears exactly once in each row
$$\sum_{j=1}^{9} x_{ijk} = 1 \quad \forall i, k \in \{1, ..., 9\}$$

#### 3. Each digit appears exactly once in each column
$$\sum_{i=1}^{9} x_{ijk} = 1 \quad \forall j, k \in \{1, ..., 9\}$$

#### 4. Each digit appears exactly once in each 3×3 submatrix
$$\sum_{i=3p+1}^{3p+3} \sum_{j=3q+1}^{3q+3} x_{ijk} = 1 \quad \forall k \in \{1, ..., 9\}, p, q \in \{0, 1, 2\}$$

#### 5. Given numbers must be preserved
$$x_{ijk} = 1 \quad \forall (i, j, k) \in G$$
where $G$ is the set of given cells with their corresponding digits.

#### 6. Binary constraint
$$x_{ijk} \in \{0, 1\} \quad \forall i, j, k \in \{1, ..., 9\}$$


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# 你截图中的数独数据
puzzle = np.array(
    [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 0, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9],
    ]
)

# 创建模型
model = gp.Model("Sudoku")
model.setParam("OutputFlag", 0)  # 不显示求解过程

# 决策变量: x[i,j,k] = 1 表示位置(i,j)填数字k
x = model.addVars(9, 9, 9, vtype=GRB.BINARY, name="x")

# 约束1: 每个格子填一个数字
for i in range(9):
    for j in range(9):
        model.addConstr(gp.quicksum(x[i, j, k] for k in range(9)) == 1)

# 约束2: 每行每个数字出现一次
for i in range(9):
    for k in range(9):
        model.addConstr(gp.quicksum(x[i, j, k] for j in range(9)) == 1)

# 约束3: 每列每个数字出现一次
for j in range(9):
    for k in range(9):
        model.addConstr(gp.quicksum(x[i, j, k] for i in range(9)) == 1)

# 约束4: 每个3x3宫每个数字出现一次
for r in range(3):
    for c in range(3):
        for k in range(9):
            model.addConstr(
                gp.quicksum(
                    x[3 * r + i, 3 * c + j, k] for i in range(3) for j in range(3)
                )
                == 1
            )

# 约束5: 固定已知数字
for i in range(9):
    for j in range(9):
        if puzzle[i, j] != 0:
            model.addConstr(x[i, j, puzzle[i, j] - 1] == 1)

# 求解
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print("Puzzle:")
    for i in range(9):
        if i % 3 == 0 and i > 0:
            print("------+-------+------")
        row = ""
        for j in range(9):
            if j % 3 == 0 and j > 0:
                row += "| "
            row += str(puzzle[i, j]) if puzzle[i, j] != 0 else "."
            row += " "
        print(row)

    print("\nSolution:")
    solution = np.zeros((9, 9), dtype=int)
    for i in range(9):
        if i % 3 == 0 and i > 0:
            print("------+-------+------")
        row = ""
        for j in range(9):
            if j % 3 == 0 and j > 0:
                row += "| "
            for k in range(9):
                if x[i, j, k].X > 0.5:
                    solution[i, j] = k + 1
                    row += str(k + 1) + " "
                    break  # Only print one digit per cell
        print(row)

    print(f"\nSolving time: {model.Runtime:.3f} seconds")
else:
    print("No solution found!")

Puzzle:
5 3 . | . 7 . | . . . 
6 . . | 1 9 5 | . . . 
. 9 8 | . . . | . 6 . 
------+-------+------
8 . . | . 6 . | . . 3 
4 . . | 8 . . | . . 1 
7 . . | . 2 . | . . 6 
------+-------+------
. 6 . | . . . | 2 8 . 
. . . | 4 1 9 | . . 5 
. . . | . 8 . | . 7 9 

Solution:
5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
------+-------+------
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
------+-------+------
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 

Solving time: 0.005 seconds
